# Network analysis
In this notebook:
- We analyze percolation of our simulation, keeping track of percolation thresh, number of components, size of the largest component.
- We explore the concept of coarse-graining our simulations (in analogy to noising). This is of interest in real world contexts
- We calculate two metrics present in Natera (2020) and Szell (2022), Directness and Coverage, to evaluate our simulations
- We calculate the impact of our simulations on the drivable network, by calculating how many roads go from two to one way.

NB to use effectively, the locations of your simulations and notebooks should be

this notebook: network_analysis.ipynb
data sets: MILANO>dataset_vehicles_preprocessed>your.data.file
simulation results: MILANO>simulations>your.simulation.results

the simulations are stored in folders formatted as such: car_lane={car_lane}>{d}>w={w}>alpha={alpha}>file.name_budget={budget}  
The car_lane variable is equal to 3.5 for all simulations that were run for the paper, but other values are acceptable.  
load path doesn't need to be specified in the budget, as the "load_data" function takes budget as its final variable.  

example load path:

load_path = os.path.join(os.getcwd(),'MILANO/simulation/car_lane=3.5/non_destructive/w=2.5/alpha=0.5)  
G = load_data(load_path,'baseG',budget)

### Imports and function definitions

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.colors as mcolors
import igraph as ig
import itertools
import json
import os
import contextily as cx
import importlib
import utils
importlib.reload(utils)

In [ ]:
ITALY_crs = 6875

## Loading data


In [ ]:
cwd = os.getcwd()
graph_path = os.path.join(cwd, "MILANO/dataset_vehicles_preprocessed/base_network.gml")
base_G = nx.read_gml(graph_path)

In [ ]:
G = base_G.copy()

In [ ]:
_, edge_attributes = utils.list_attributes(G)
if('p_w' in edge_attributes):
   print('p_values already calculated')
else:
   utils.p_values(G)

## 1) Percolation analysis
percolation thresholds and size of gcc for single alphas and widths   

This is a series of functions written specifically to look at percoaltion, over a range of budgets for a range of alphas.  
percolation results are returned.  
Further below you can find a function that doesn't store results, but introduces a form of noise to the simulations.

### Example

In [ ]:
# the example only works if you have completed the simulations in the last notebook

alpha_list = [0, 0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
budget_list = list(np.arange(20,200,10))+ list(np.arange(200, 1000, 40))
w = 2.5
load_path_template = os.path.join(cwd, 'MILANO/simulations/car_lane=3.5/non_destructive/w={w}/alpha={alpha}')
gcc_sizes, num_components, gcc_derivatives, percolation_thresholds = utils.calc_percolation(alpha_list, budget_list, load_path_template, w)

### Plot

In [ ]:
fig = utils.plot_gcc_and_components(alpha_list, budget_list, gcc_sizes, num_components)
# fig.savefig(f'figures/gcc_and_components_width={w}.png', dpi = 200, bbox_inches = 'tight')

In [ ]:
utils.save_percolation_results(gcc_sizes, num_components, gcc_derivatives, percolation_thresholds, 'percolation_analysis')

In [ ]:
gcc_sizes, num_components, gcc_derivatives, percolation_thresholds = utils.load_percolation_results('percolation_analysis')

## 2) (Optional) Noising simulations: 
widths and betweennesses have few identical entries, but in practical contexts any ranking of roads by width or betweenness would have some coarse graining within it. These functions coarse grain the rankings and add a random selection element to them, allowing us to average over any number of simulations.  
NB code is written so that baseG is loaded inside the calc function, not subG. simulations are "redone" in this case

### Example


In [ ]:
#example usage
alpha_list = [0, 0.1, 0.3, 0.5, 0.7, 0.9, 1]
budget_list = list(np.arange(1,20,20)) + list(np.arange(20,200,10))+ list(np.arange(200, 1000, 40))
w = 2.5
load_path_template = os.path.join(cwd, 'MILANO/simulations/car_lane=3.5/non_destructive/w={w}/alpha={alpha}')
Gcc_sizes_dict, N_components_dict, var_Gcc_sizes_dict, var_N_components_dict = utils.noise_simulations(alpha_list, budget_list, load_path_template, w, n = 50)
#save_noise_simulations(Gcc_sizes_dict, N_components_dict, var_Gcc_sizes_dict, var_N_components_dict, 'noise_simulations')


In [ ]:
utils.save_noise_simulations(Gcc_sizes_dict, N_components_dict, var_Gcc_sizes_dict, var_N_components_dict, 'noise_simulations')

In [ ]:
fig = utils.plot_noised(Gcc_sizes_dict, N_components_dict, var_Gcc_sizes_dict, var_N_components_dict, alpha_list)


## 3) natera and szell metrics for evaluating simulations
directness, coverage



### i) directness.  
it quantifies the average distance on the base network between two nodes over the average distance on the sub network.  
for example if the average distance between two nodes on the subnetwork is 1.5 times the average distance on the base network, the directness is 0.666.  
but how do you handle a pair of nodes that is only reachable from getting on the car network?  
From what i understand, natera says "if A and B are not connected by the bike network, then d_bike = infinity, so directness = d_car/d_bike --> 0".
sounds kind of brusque if you ask me but ok. They generate 1000 nodes and check all the distances.

ideally my metric would take all pairs of nodes on the network, calculate the shortest path on the base network,
and then the shortest path that stays on the subnetwork for as long as possible, and then calcs the ratio.  

for example given A and B on the base network, the shortest path is easily calculated.  
to get the shortest path while passing through the subnetwork i need to divde the path:
first check if A is on the subnet, and on which component. if it not, find closest point of the subnet to A, and call it A_k, where k is the number of the component of the subnet it is on.  If it is, just label it as A_k
then check if B is on the subnet. if it not, find closest point of the subnet to B, and call it B_j, where j is the cnumber of the component of the subnet it is on.  
if k == j calc shortest path on the subnet.  
else 
store lengths AA' and BB',

### Example


In [ ]:
#NB takes a long time to run

alpha_list = [0, 0.1, 0.3 ,0.5, 0.7, 0.9, 1.0
              ]
budget_list = list(np.arange(20, 200, 10)) + list(np.arange(200, 1000, 40))
w = 2.5
load_path_template = os.path.join(cwd, 'MILANO/simulations/car_lane=3.5/non_destructive/w={w}/alpha={alpha}')
directness_dict = utils.calc_directness(alpha_list, budget_list, load_path_template, w, nsamples=1000, weight='length')
#save_directness(directness_dict, 'directness_dict.json')
fig = utils.plot_directness(directness_dict, budget_list, alpha_list)

In [ ]:
utils.save_directness(directness_dict, 'directness_dict.json')

### ii) Coverage
Coverage: the definition of coverage according to Szell is: area of all grown structures + 500m buffer. I can't do that because i don't have the gdf of my networks. However, it's high time that i write a function that takes in the graph and gives the corresponding gdf from my original data set. it shouldn't be too hard, since i have edge id's from the edgelist


### Example

In [ ]:
# take about 3 minutes to run
alpha_list = [0, 0.1, 0.3, 0.5, 0.7, 0.9, 1.0
              ]
budget_list = list(np.arange(20, 200, 10)) + list(np.arange(200, 1000, 40))
w = 2.5
load_path_template = os.path.join(cwd, 'MILANO/simulations/car_lane=3.5/non_destructive/w={w}/alpha={alpha}')
dissolved_gdf = gpd.read_file(os.path.join(cwd,'MILANO/dataset_vehicles_preprocessed/dissolved_roads.gpkg'))
coverage_dict = utils.calc_coverages(alpha_list, budget_list, load_path_template, w, dissolved_gdf)
fig = utils.plot_coverages(coverage_dict, alpha_list, budget_list)

In [ ]:
utils.save_coverages(coverage_dict, 'coverage_dict.json')

## 4) Car lane preservation after simulations
To estimate the impact of bike lane construction on automobile traffic, we would need to simulate car flow.  
In the absence of this, we can look at how the structure of the network itself may impact automobiles.  
The amount of one way edges in a network is connected to the creation of jams and congestion (Carmona et al., 2020).  
we use the Italian transport code (codice della strada) to say that all lanes must be at least 2.75m wide on residential roads, and at least 3.5m wide on primary roads.  
With this in mind, all roads with a width smaller than 5.5m can absolutely not be two way streets, and more in general two way streets will usually be at least 7m wide.  
We assume all streets wider than 7m to be two way streets, and all the ones narrower than 7m to be one way.  
Therefore, measuring the amount of streets going from wider to narrower than 7m before and after our simulations is a measure of impact on the drivable network.


### Example

In [ ]:
alpha_list = [0, 0.1, 0.3 ,0.5, 0.7, 0.9, 1.0]
budget_list = list(np.arange(20, 200, 10)) + list(np.arange(200, 1000, 40))
w = 2.5
load_path_template = os.path.join(cwd, 'MILANO/simulations/car_lane=3.5/non_destructive/w={w}/alpha={alpha}')
impacted_edges_dict = utils.calc_car_impact(budget_list, alpha_list, load_path_template, w = 2.5)
fig = utils.plot_car_impact(impacted_edges_dict, alpha_list, budget_list)

In [ ]:
utils.save_car_impact(impacted_edges_dict, 'impacted_edges_dict.json')

## 5) Plotting parameter space
in this cell we plot parameter space (alpha, budget) and make two color maps of directness and of preserved/impacted lanes respectively. 
We want budget on the x axis, $\alpha$ on the y axis, and colors for the value of the observable.  
Our results dicts have $\alpha$ as keys, and lists of measurements for each budget value as values.  
We first convert them to dataframes, then plot

In [ ]:

budget_list = list(np.arange(20, 200, 10)) + list(np.arange(200, 1000, 40))
directness_dict = utils.load_directness('directness_dict.json')
impacted_edges_dict = utils.load_car_impact('impacted_edges_dict.json')

# convert directness_dict and car impact dict to df
directness_df = utils.dict_to_df(directness_dict, budget_list)
impacted_edges_df = utils.dict_to_df(impacted_edges_dict, budget_list)
# Plot the colored maps
fig1 = utils.plot_colored_map(directness_df, attribute='Directness')
fig2 = utils.plot_colored_map(impacted_edges_df, attribute = 'Impacted Edges (%)')
# Save the figures
fig1.savefig('directness_colored_map.png', dpi=300, bbox_inches='tight')
fig2.savefig('impacted_edges_colored_map.png', dpi=300, bbox_inches='tight')

## 6) mapping impacted streets
we know that, since our simulations apply a 2.5m bike lane to affected edges, than those counted as impacted, i.e. going from more than 7m wide to less than 7m wide, will all have width between 7m and 9.5m in the base graph.  
We show the potentially impacted edges on a map, and then color them by score to see which ones would be impacted first.  


In [ ]:
# let's check what the plot suggests for alpha = 1.0
# it seems there are 600 km of roads wider than 9.5m. let's verify this
G1 = base_G.copy()
edge1 = nx.to_pandas_edgelist(G1, edge_key='key')
edge1 = edge1[edge1['width'] > 9.5]
edge1['cumlength'] = edge1['length_for_bc'].cumsum()
total_length = edge1['cumlength'].iloc[-1] / 1000  # Convert to km
print(f'Total length of edges with width > 9.5m: {total_length:.2f} km')

In [ ]:
# Let's see what the plot suggests for alpha = 0.0
# Take the roads from the base graph and rank them by p_bc. If we look at the first 200 km of roads,
# we should find that about 4% of them have a width between 7 and 9.5m
G1 = base_G.copy()
utils.p_values(G1)
edge1 = nx.to_pandas_edgelist(G1, edge_key='key')
edge1 = edge1.sort_values('p_bc', ascending=False).reset_index(drop=True)
edge1['cumlength'] = edge1['length_for_bc'].cumsum()
# total_length = edge1['cumlength'].iloc[-1] / 1000  # Convert to km
edge1 = edge1[edge1['cumlength'] < 200000]  # Keep only the first 200 km
temp = edge1[edge1['width'] > 7]
temp = temp[temp['width'] < 9.5]
frac_length = len(temp) / len(G1.edges) * 100
print(f'Fraction of edges with width between 7 and 9.5m in the first 200 km: {frac_length:.2f}%')

let's look at the graph now. in particular i'd like to see the betweenness distribution of the streets in this graph

In [ ]:
edgelist = nx.to_pandas_edgelist(G, edge_key='key')
# only keep edges with width between 7 and 9.5m
possible_impacted = edgelist[(edgelist['width'] >= 7) & (edgelist['width'] <= 9.5)]
# create iterable of these edges
edge_iterable = set(map(tuple, possible_impacted[['source', 'target', 'key']].values))
# create subgraph with these edges
impactable_G = G.edge_subgraph(edge_iterable).copy()

If we want to see the probability of these edges being part of a simulation for a given alpha, we can:  
- choose a value for $\alpha$
- calculate the score of the edges as $s = \alpha p_{w} + (1-\alpha) p_{bc}$
- plot the edges their scores colormapped  

The case where $\alpha = 0$ corresponds to a full betweenness strategy, whereas the case where $\alpha = 1$ is a full width strategy 

In [ ]:
utils.plot_impactable_edges(0, impactable_G, save = True, width = 1, dpi = 200)  

In [ ]:
alpha = 0
budget = 200
fig = utils.plot_impacted_edges(alpha, budget, impactable_G, color = 'red', save = False, width = 1, dpi = 300)
# plt.savefig(f'impacted_edges_alpha={alpha}_budget={budget}.png', dpi = 300, bbox_inches='tight')

## 7) Directness compared to preserved edges for varying alpha


In [ ]:

fig = utils.plot_directness_and_car_impact(alpha_list, 400, directness_dict, impacted_edges_dict)
fig.savefig('directness_and_car_impact_budget=400.png', dpi=300)

## Future work

### Efficiency Calculation 
As introduced by Latora (2001), but also carried out in Szell (2022).  
Remember, efficiency is a measure of the speed of information transmission throughout the network (i.e flow).  
When a network is a small world, the average paths between pairs are short, but the networks are also highly clustered.  
We can define efficiency (global) as the average of the inverse shortest distance between nodes i and j, i.e  
$\epsilon_{ij} = 1/d_{ij}$,  
and $E_{glob} = \sum_{i\ne j} \frac{\epsilon_{ij}}{N(N-1)}$.   
Note that a fully connected network (all nodes connected by edges) would have a maximum possible efficiency 
We can also define local efficiency as the average for all nodes of the global efficiencies of all subgraphs-of-neighbors:  
$E_{loc} = \sum_{i = 1}^N \frac{1}{N} E_{glob}(G_i)$  
where $G_i$ is the graph of the neighbors of $i$. This measures the connectivity of $i$'s neighbors if i is not present, i.e. it's a measure of resilience and fault tollerance.  

In general, a good transportation network should have good global efficiency, making it easy to travel across the network as a whole, whereas local efficiency can be a desirable but unneccesary trait (see subway networks).  

to calculate network global efficiency:  
- calculate all shortest paths between nodes (n.b if no path from i to j, make distance infinity).
- sum the inverses (inverse of infinity is 0) and divide by N(N-1) for pairs of nodes to get efficiency.
- calculate normalization: fully connected graph --> all physical distances between nodes
- normalize


### first/last portion calculation
we want to calculate how much time is spent inside the subnetwork for the average trip.
ideas: function to find all pairs of nodes on baseG. routing function that calculates list of edges from A to B (possibly weighted, if not just sum weights of list after); obviously using shortest path. Calc lengths of edges in subG and save result.
length(subG edges)/length(baseG edges) is our number for each pair A B (or even un normalized). average of all these is our result


i can't use cugraph with 'all_pairs_shortest_path' algo because it doesn't supported weighted graphs (as of july 2024)  
I can use igraph.distances